In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from rasterio.crs import CRS

import urllib3
import os

In [4]:
url_template = "https://api.water.usgs.gov/nldi/linked-data/nwissite/USGS-{site_no}/basin?f=json"
stream_gage_fp = os.path.relpath("../datasets/stream_gages/watershed_stream_gages.csv")
site_watersheds_out = os.path.relpath("../datasets/stream_gages/site_watersheds.geojson")

In [5]:
stream_gages = pd.read_csv(stream_gage_fp, dtype={"site_no": str})
# stream_gages.info()
features = []
site_nos = []
basin_names = []
for gage in stream_gages.loc[:, ["site_no", "BASIN"]].itertuples():
  resp = urllib3.request("GET", url_template.format(site_no=gage.site_no))
  if resp.status == 200:
    site_nos.append(gage.site_no)
    basin_names.append(gage.BASIN)
    features += resp.json()["features"]
  # print(resp.status, resp.json())
print(features)

[{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-74.090246751, 42.259145501], [-74.085239288, 42.258180601], [-74.08232545, 42.252991302], [-74.07366207, 42.246370641], [-74.070027353, 42.239714438], [-74.070912344, 42.234185434], [-74.064643654, 42.225251771], [-74.072095309, 42.224371997], [-74.093859306, 42.227058836], [-74.100736709, 42.224026045], [-74.112040138, 42.225877444], [-74.117150275, 42.224989722], [-74.121553665, 42.226624799], [-74.138493355, 42.224286945], [-74.140021081, 42.222812233], [-74.151240889, 42.22505359], [-74.167788319, 42.222317679], [-74.171547162, 42.225957141], [-74.187948144, 42.230456327], [-74.196654714, 42.231266304], [-74.202308496, 42.235376437], [-74.220886544, 42.236235142], [-74.232356007, 42.238474809], [-74.238377386, 42.237391679], [-74.250119127, 42.243985427], [-74.254306754, 42.24464559], [-74.258787424, 42.241758733], [-74.270344935, 42.239212216], [-74.278570096, 42.241281233], [-74.283797457, 42.244492206], [-74

In [57]:
gdf = gpd.GeoDataFrame.from_features(features=features, crs=CRS.from_epsg(4326)).to_crs(CRS.from_epsg(6537))
gdf["site_no"] = site_nos
gdf["basin_name"] = basin_names
gdf["area_sq_miles"] = gdf.area * 3.58701 * (10 ** -8)

In [58]:
gdf.explore()

In [59]:
gdf.to_file(site_watersheds_out)

In [60]:
site_watersheds_gdf = gpd.read_file(site_watersheds_out)
site_watersheds_gdf.explore()